### Collab

#### git clone

In [1]:
#!git clone https://github.com/<your name>/<your project>.git
#pip install -r requirements.txt

### Importamos las librerias y el dataset

In [2]:
import numpy as np
import os
import shutil
import datetime

In [3]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Embedding, LSTM, Dense
from keras.callbacks import TensorBoard
from keras.preprocessing.text import one_hot
import pandas as pd

Using TensorFlow backend.


In [4]:
from libs.workshop_libs.tensorboard import TrainValTensorBoard

### Importar el dataset

In [5]:
#Ugly Example preprocessing
def encodeData(data,vocab_size = 5000):
    data_out=[]
    for index, row in data.iterrows():
        text=(row.iloc[0]+' ### '+row.iloc[1]+' ### '+row.iloc[2])
        data_out.append(one_hot(text, vocab_size))
    return np.array(data_out)
def encodedLabels(data):
    data_out=[]
    for index, row in data.iterrows():
        if row.iloc[0] == 'others':
            data_out.append([1,0,0,0])
        elif row.iloc[0] == 'happy':
            data_out.append([0,1,0,0])
        elif row.iloc[0] == 'angry':
            data_out.append([0,0,1,0])
        elif row.iloc[0] == 'sad':
            data_out.append([0,0,0,1])
        else:
            raise Exception()
    return np.array(data_out).astype(int)
def prepare_data(x_data,max_conv_length):
    x_data = sequence.pad_sequences(x_data, maxlen=max_conv_length)
    return x_data
def loadAndPrepareData(fname,vocab_size,max_conv_length=50):
    data = pd.read_csv(fname,sep='	',header=None)
    x_data_raw=data.filter([1,2,3], axis=1).copy(deep=True)
    y_data_raw=data.filter([4], axis=1).copy(deep=True)  
    x_data = encodeData(x_data_raw,vocab_size)
    y_data = encodedLabels(y_data_raw)
    x_data = prepare_data(x_data,max_conv_length)                       
    return x_data,y_data

In [6]:
vocab_size=5000
max_conv_length=50

In [7]:
x_train,y_train=loadAndPrepareData('data_train/train.txt',vocab_size,max_conv_length)
x_dev,y_dev=loadAndPrepareData('data_dev/dev.txt',vocab_size,max_conv_length)

In [8]:
len(x_train[0])

50

In [9]:
x_train[0][:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [10]:
len(x_train)

30160

In [11]:
#QUITAR
max_data=10000
x_train=x_train[0:max_data]
y_train=y_train[0:max_data]
x_dev=x_dev[0:max_data]
y_dev=y_dev[0:max_data]

### Revisemos el dataset

In [12]:
## Cargamos tensorboard

In [13]:
%load_ext tensorboard
#%reload_ext tensorboard

In [14]:
#!kill 4430

In [15]:
#!ps aux | grep -i tb

In [16]:
logs_base_dir = "./logs"
print(shutil.rmtree(logs_base_dir))

None


In [17]:
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

Reusing TensorBoard on port 6006 (pid 6823), started 1:37:35 ago. (Use '!kill 6823' to kill it.)

In [18]:
### Uniformizemos los datos, truncados y padding

In [19]:
def create_model(top_words,max_review_length):
    embedding_vector_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(4, activation='sigmoid'))
    print(model.summary())
    return model

In [20]:
def train_model(model,x_train,y_train,x_dev,y_dev):
        logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']
                     )
        model.fit(x_train,
                  y_train,
                  batch_size=64, #64,
                  validation_data=(x_dev, y_dev),
                  nb_epoch=10,
                  callbacks=[TrainValTensorBoard(logdir, 
                                                 histogram_freq=1,
                                                 write_graph=True
                                                )]
                 )

In [21]:
print(x_train.shape)
print(x_dev.shape)

(10000, 50)
(2755, 50)


In [22]:
model=create_model(vocab_size,max_conv_length)

W0720 18:44:30.807185 140308776519488 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0720 18:44:30.818528 140308776519488 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0720 18:44:30.820264 140308776519488 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 32)            160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 213,604
Trainable params: 213,604
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
train_model(model,x_train,y_train,x_dev,y_dev)

W0720 18:44:31.044564 140308776519488 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0720 18:44:31.060983 140308776519488 deprecation_wrapper.py:119] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

/home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
W0720 18:44:31.491600 140308776519488 deprecation.py:323] From /home/leo/ETH/DS3/gtWorkshop/sandbox/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.wh

Train on 10000 samples, validate on 2755 samples
Epoch 1/10
10000/10000 [==============================] - 6s 552us/step - loss: 1.1285 - acc: 0.5395 - val_loss: 0.5160 - val_acc: 0.8682
Epoch 2/10
10000/10000 [==============================] - 5s 500us/step - loss: 0.5771 - acc: 0.7897 - val_loss: 0.3835 - val_acc: 0.8726
Epoch 3/10
10000/10000 [==============================] - 5s 524us/step - loss: 0.4053 - acc: 0.8557 - val_loss: 0.4150 - val_acc: 0.8523
Epoch 4/10
10000/10000 [==============================] - 5s 530us/step - loss: 0.3331 - acc: 0.8834 - val_loss: 0.4528 - val_acc: 0.8345
Epoch 5/10
10000/10000 [==============================] - 6s 567us/step - loss: 0.2908 - acc: 0.9002 - val_loss: 0.4414 - val_acc: 0.8428
Epoch 6/10
10000/10000 [==============================] - 6s 563us/step - loss: 0.2524 - acc: 0.9129 - val_loss: 0.5059 - val_acc: 0.8189
Epoch 7/10
10000/10000 [==============================] - 7s 668us/step - loss: 0.2254 - acc: 0.9244 - val_loss: 0.4581 - v

### Evaluamos el modelo

In [25]:
scores = model.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2755/2755 [==============================] - 0s 166us/step
Accuracy: 82.25%


### Guardamos el modelo

In [26]:
outpath='models/'
os.makedirs(outpath, exist_ok=True)

In [27]:
model.save(outpath+"model.h5")

In [28]:
# serialize model to JSON
model_json = model.to_json()
with open(outpath+"model.json", "w") as json_file:
    json_file.write(model_json)

In [29]:
from keras.models import load_model

In [30]:
model2=load_model("models/model.h5")

In [33]:
scores = model2.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

2755/2755 [==============================] - 1s 209us/step
Accuracy: 82.25%


In [35]:
print(model.metrics_names)

['loss', 'acc']


### ADDENDUM: One hot encoder

In [ ]:
from keras.preprocessing.text import one_hot

In [ ]:
texto=["Esta es una frase","Otra frase mas larga", "todo esto viene de nuestro dataset de interes"]

In [ ]:
vocab_size = 50
palabras_vectorizadas = [one_hot(d, vocab_size) for d in texto]
print(palabras_vectorizadas)

### Problemas con numpy nuevos y keras

In [ ]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
#(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
# restore np.load for future normal usage
np.load = np_load_old

In [ ]:
### Otra solucion

In [ ]:
#!pip uninstall numpy

In [ ]:
#!pip install numpy==1.16.1